# YOLOv8 Accuracy vs. Inference Time Analysis

This notebook analyzes the trade-off between accuracy (mAP) and inference time for different YOLOv8 models (nano, small, medium, large, xlarge) using the 'Food Image Segmentation' dataset.

In [6]:
# Install Ultralytics (if not already installed)
!pip install ultralytics

import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.7.0+cu126
CUDA available: True


## 1. Define Models and Dataset Path

In [ ]:
models = {
    # 'yolov8n': 'yolov8n.pt',  # nano
    # 'yolov8s': 'yolov8s.pt',  # small
    # 'yolov8m': 'yolov8m.pt',  # medium
    'yolov8l': 'yolov8l.pt',  # large
    'yolov8x': 'yolov8x.pt'   # xlarge
}

data_yaml_path = '/home/intercogni/github/intercogni/yolov8-vs-rtdetr/data/yolov8/data.yaml'
imgsz = 640 # Image size for evaluation

In [8]:
trained_models = {}

In [9]:
trained_models

{}

In [10]:
for model_name, weights_path in models.items():
  print(f"\nTraining {model_name} model...")
  try:
    # Load YOLO model with pretrained weights
    model = YOLO(weights_path)
    
    # Train the model on the dataset declared in data_yaml_path.
    # Adjust epochs and other parameters as needed.
    model.train(data=data_yaml_path, imgsz=imgsz, epochs=9999999, device=0 if torch.cuda.is_available() else 'cpu', patience=16)
    
    # Store the trained model in a variable for later validation
    trained_models[model_name] = model
    
  except Exception as e:
    print(f"Error training {model_name}: {e}")


Training yolov8s model...
New https://pypi.org/project/ultralytics/8.3.150 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/intercogni/github/intercogni/yolov8-vs-rtdetr/data/yolov8/data.yaml, epochs=9999999, time=None, patience=16, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment

train: Scanning /home/intercogni/github/intercogni/yolov8-vs-rtdetr/data/yolov8/train/labels.cache... 7334 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7334/7334 [00:00<?, ?it/s]
val: Scanning /home/intercogni/github/intercogni/yolov8-vs-rtdetr/data/yolov8/valid/labels.cache... 699 images, 0 backgrounds, 0 corrupt: 100%|██████████| 699/699 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 9999999 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  1/9999999      7.32G      1.623       3.05      1.891         25        640: 100%|██████████| 459/459 [00:35<00:00, 12.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:01<00:00, 12.69it/s]

                   all        699       2164      0.696      0.559      0.615      0.349


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  7.33it/s]


                   all        699       2164       0.98      0.982      0.989      0.871
               AW cola         17         17      0.942          1      0.995      0.881
          Beijing Beef         53         53          1          1      0.995      0.963
             Chow Mein         10         10      0.979          1      0.995       0.98
            Fried Rice         31         31      0.994          1      0.995      0.924
             Hashbrown         15         15      0.988          1      0.995      0.815
   Honey Walnut Shrimp         23         23      0.988          1      0.995       0.96
      Kung Pao Chicken         63         65      0.997          1      0.995      0.956
String Bean Chicken Breast         40         40      0.995          1      0.995      0.945
          Super Greens         11         11      0.983          1      0.995      0.951
The Original Orange Chicken         26         26      0.992          1      0.995      0.923
    White St

train: Scanning /home/intercogni/github/intercogni/yolov8-vs-rtdetr/data/yolov8/train/labels.cache... 7334 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7334/7334 [00:00<?, ?it/s]
val: Scanning /home/intercogni/github/intercogni/yolov8-vs-rtdetr/data/yolov8/valid/labels.cache... 699 images, 0 backgrounds, 0 corrupt: 100%|██████████| 699/699 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 


: 

## 2. Evaluate Models

In [3]:
results_data = []

for name, weights_path in models.items():
    print(f"\nEvaluating {name}...")
    try:
        # Load a pretrained YOLOv8 model
        model = YOLO(weights_path)

        # Validate the model on the dataset
        # The 'metrics' object contains mAP50-95, mAP50, and inference time
        metrics = model.val(data=data_yaml_path, imgsz=imgsz, conf=0.25, iou=0.6, device=0 if torch.cuda.is_available() else 'cpu')

        # Extract relevant metrics
        map50_95 = metrics.box.map # mAP50-95
        map50 = metrics.box.map50 # mAP50
        inference_time_ms = metrics.speed['inference'] # Inference time in ms

        results_data.append({
            'Model': name,
            'mAP50-95': map50_95,
            'mAP50': map50,
            'Inference Time (ms)': inference_time_ms
        })
        del model # Clear model from memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"Error evaluating {name}: {e}")
        results_data.append({
            'Model': name,
            'mAP50-95': None,
            'mAP50': None,
            'Inference Time (ms)': None
        })

df_results = pd.DataFrame(results_data)
print("\nEvaluation Results:")
print(df_results)


Evaluating yolov8n...
Ultralytics 8.3.40 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
Error evaluating yolov8n: 
Dataset 'data/yolov8/data.yaml' images not found ⚠️, missing path '/home/intercogni/data/yolov8/valid/images'
Note dataset download directory is '/home/intercogni'. You can update this in '/home/intercogni/.config/Ultralytics/settings.json'

Evaluating yolov8s...


100%|█████████████████████████████████████████████████| 21.5M/21.5M [00:00<00:00, 69.9MB/s]

Ultralytics 8.3.40 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)


YOLOv8s summary (fused): 168 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
Error evaluating yolov8s: 
Dataset 'data/yolov8/data.yaml' images not found ⚠️, missing path '/home/intercogni/data/yolov8/valid/images'
Note dataset download directory is '/home/intercogni'. You can update this in '/home/intercogni/.config/Ultralytics/settings.json'

Evaluating yolov8m...


100%|█████████████████████████████████████████████████| 49.7M/49.7M [00:00<00:00, 66.0MB/s]


...

100%|█████████████████████████████████████████████████| 83.7M/83.7M [00:01<00:00, 78.0MB/s]

Ultralytics 8.3.40 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)


YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs
Error evaluating yolov8l: 
Dataset 'data/yolov8/data.yaml' images not found ⚠️, missing path '/home/intercogni/data/yolov8/valid/images'
Note dataset download directory is '/home/intercogni'. You can update this in '/home/intercogni/.config/Ultralytics/settings.json'

Evaluating yolov8x...


100%|███████████████████████████████████████████████████| 131M/131M [00:01<00:00, 82.5MB/s]


Ultralytics 8.3.40 🚀 Python-3.11.12 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16064MiB)
YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs
Error evaluating yolov8x: 
Dataset 'data/yolov8/data.yaml' images not found ⚠️, missing path '/home/intercogni/data/yolov8/valid/images'
Note dataset download directory is '/home/intercogni'. You can update this in '/home/intercogni/.config/Ultralytics/settings.json'

Evaluation Results:
     Model mAP50-95 mAP50 Inference Time (ms)
0  yolov8n     None  None                None
1  yolov8s     None  None                None
2  yolov8m     None  None                None
3  yolov8l     None  None                None
4  yolov8x     None  None                None


## 3. Plot Results

In [ ]:
plt.figure(figsize=(10, 7))
sns.scatterplot(
    data=df_results,
    x='Inference Time (ms)',
    y='mAP50-95',
    hue='Model',
    s=200, # Size of points
    style='Model', # Different markers for different models
    markers=True
)

# Add model names next to points
for i, row in df_results.iterrows():
    plt.text(row['Inference Time (ms)'] + 0.5, row['mAP50-95'], row['Model'], ha='left', va='center', fontsize=10)

plt.title('YOLOv8 Accuracy (mAP50-95) vs. Inference Time')
plt.xlabel('End-to-end Latency T4 TensorRT FP16 (ms)') # Matching the provided image's x-axis label
plt.ylabel('COCO AP (%)') # Matching the provided image's y-axis label
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 4. Analysis

In [ ]:
"""
Based on the generated plot and evaluation results, we can observe the following:

- **Trade-off**: There is a clear trade-off between model accuracy (mAP) and inference time. Generally, larger models (e.g., yolov8x) achieve higher accuracy but require more inference time, while smaller models (e.g., yolov8n) are faster but less accurate.

- **Model Performance**: 
    - `yolov8n` (nano) is the fastest but has the lowest mAP.
    - As we move from `yolov8s` to `yolov8m`, `yolov8l`, and `yolov8x`, the mAP generally increases, but so does the inference time.

- **Choosing a Model**: The choice of model depends on the specific application's requirements for speed versus accuracy. For real-time applications where speed is critical, `yolov8n` or `yolov8s` might be preferred. For applications requiring higher accuracy, `yolov8l` or `yolov8x` would be more suitable, provided the inference time is acceptable.

This analysis provides a quantitative understanding of how different YOLOv8 model sizes perform on the given dataset, aiding in model selection for deployment.
"""